In [4]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import torch
from torch.utils.data import DataLoader

In [6]:
from torchvision import datasets , models , transforms
from torchsummary import summary
import matplotlib.pyplot as plt
from PIL import Image

ModuleNotFoundError: No module named 'torchvision'

In [1]:
data_path = '\data_input\image'
dataset = datasets.ImageFolder(data_path)
num_samples = len(dataset)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Seni\AppData\Local\Temp\ipykernel_22516\2631649728.py:1: SyntaxWarning: invalid escape sequence '\d'
  data_path = '\data_input\image'


## preprocess

In [ ]:
TRAIN_RATIO , VALID_RATIO = 0.8 , 0.1
n_train_examples = int( num_samples * TRAIN_RATIO )
n_valid_examples = int( num_samples * VALID_RATIO )
n_test_examples = num_samples - n_train_examples - n_valid_examples

train_dataset , valid_dataset , test_dataset = dataset.random_split( dataset ,
                                                                     [n_train_examples, n_valid_examples, n_test_examples]
)

In [ ]:
IMG_SIZE = 224
train_transforms = transforms.Compose ([
    transforms.Resize((IMG_SIZE, IMG_SIZE ) ) ,
    transforms.RandomHorizontalFlip() ,
    transforms.RandomRotation(0.2) ,
    transforms.ToTensor() ,
    transforms.Normalize([0.5 , 0.5 , 0.5], [0.5 , 0.5 , 0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE , IMG_SIZE ) ) ,
    transforms.ToTensor() ,
    transforms.Normalize([0.5 , 0.5 , 0.5] , [0.5 , 0.5 , 0.5])
])

train_dataset.dataset.transform = train_transforms
valid_dataset.dataset.transform = test_transforms
test_dataset.dataset.transform = test_transforms

In [ ]:
BATCH_SIZE = 256

train_dataloader = dataset.DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True
)
valid_dataloader = dataset.DataLoader(
    valid_dataset, 
    batch_size=BATCH_SIZE 
)
test_dataloader = dataset.DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE
)

In [ ]:
def build_model(input_1, input_2, input_3):
    cnn_input = Input(shape=(input_1, input_2, input_3), name='CNN Input')
    cnn1 = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(cnn_input)
    cnn1 = BatchNormalization()(cnn1)
    cnn1 = MaxPooling2D(pool_size=2)(cnn1)
    cnn2 = Conv1D(filters=16, kernel_size=(3,3), activation='relu')(cnn1)
    cnn2 = BatchNormalization()(cnn2)
    cnn2 = MaxPooling2D(pool_size=2)(cnn2)
    ft = Flatten()(cnn2)
    dns1 = Dense(64, activation='relu')(ft)
    dns2 = Dense(32, activation='relu')(dns1)
    dns3 = Dense(32, activation='relu')(dns2)
    output1 = Dense(1, name='length', activation = 'relu')(dns3)
    output2 = Dense(1, name='width', activation = 'relu')(dns3)
    output3 = Dense(1, name='taillength', activation = 'relu')(dns3)
    model = Model(inputs=cnn_input, outputs=[output1, output2, output3])
    optimizer = Adam(0.0001)
    model.compile(optimizer=optimizer, loss={'length': 'mae', 'width': 'mae', 'taillength': 'mae'})
return model
Then you can call build_model using height, width and channel lenght of your input images.
The training with be something like model.fit(X, [y_0, y_1, y_2], epochs = your choice, batch_size = your choice, validation_data = ([Xval], [yval_0, yval_1, yval_2]))